# Data Merge & Cleaning
In this notebook, I will merge all data and check what's missing. 
I'll also use this section to figure out what kind of preprocessing will be necessary and how to extract text information.

In [1]:
import pandas as pd
import numpy as np

import pickle

In [2]:
harvard1 = pd.read_pickle('PKL/raw_data_Harvard_1.pkl')

In [3]:
harvard2 = pd.read_pickle('PKL/raw_data_Harvard_3.pkl')

In [4]:
risd1 = pd.read_pickle('PKL/raw_data_RISD_1.pkl')

In [5]:
risd2 = pd.read_pickle('PKL/raw_data_RISD_2.pkl')

In [6]:
moma = pd.read_csv('DATA/MoMA_data.csv')

## Harvard Museum
---
Let's look at the Harvard data first.

In [7]:
harvard = pd.concat([harvard1, harvard2], axis = 0)

In [8]:
# remove duplicates
harvard = harvard.drop_duplicates(subset = ['id'])

### missing image
Drop if it's missing image url.

In [9]:
harvard = harvard.dropna(subset = ['primaryimageurl'])

### Feature Consolidation
I'll consolidate scattered info into ...
1. period
2. culture
3. medium
4. title
5. description
6. palette
7. date
8. name of the artist

### Unnesting Subcategories
There are sub-categories, which I'll unnest first.

Images, worktypes, colors, people are dictionary nested in the list. We'll need to extract information from them.

In [19]:
def extract_info(x, name):
    ''' 
    INPUT: a list of dictionary, key name
    OUTPUT: key value, consolidated as one string if many
    '''
    if isinstance(x, list):
        if len(x) == 1:
            return x[0][name]
        else:
            inst = []
            for i in range(len(x)):
                val = x[i][name]
                if val not in inst and val != None:
                    inst.append(val)
            if len(inst) > 1:
                return ', '.join(inst)
            elif len(inst) == 1:
                return inst[0]
            else:
                return None
    else:
        return None

In [20]:
# for image info
imageinfo = ['description', 'alttext', 'publiccaption']
for item in imageinfo: 
    harvard[f'img_{item}'] = harvard['images'].apply(lambda x: extract_info(x, item))

In [21]:
# worktype
harvard['worktype'] = harvard['worktypes'].apply(lambda x: extract_info(x, 'worktype'))

In [22]:
# color
harvard['color'] = harvard['colors'].apply(lambda x: extract_info(x, 'hue'))

In [23]:
# for artist, do the same but only take the artist roles
def extract_info_artist(x, name):
    if isinstance(x, list):
        if len(x) == 1:
            return x[0][name]
        else:
            inst = []
            for i in range(len(x)):
                if x[i]['role'] == 'Artist':
                    val = x[i][name]
                    if val not in inst and val != None:
                        inst.append(val)
            if len(inst) > 1:
                return ', '.join(inst)
            elif len(inst) == 1:
                return inst[0]
            else:
                return None
    else:
        return None

In [24]:
harvard['artist'] = harvard['people'].apply(lambda x: extract_info(x, 'displayname'))

In [25]:
harvard = harvard.drop(['images', 'worktypes', 'colors', 'people'], axis = 1)

### Combining columns
I'll combine columns in this manner.
1. period: if datebegin is 0 use century
2. description: combine everything: 'style', 'commentary', 'description', 'labeltext', 'img_description', 'img_alttext', 'img_publiccaption'

#### Period

In [28]:
harvard['century'] = np.where(harvard['century'].isnull(), harvard['dated'], harvard['century'])
harvard['period'] = np.where(harvard['period'].isnull(), harvard['century'], harvard['period'])
harvard['period'] = np.where(harvard['datebegin'] == 0, harvard['period'], harvard['datebegin'])

#### Description

In [29]:
desc_list = ['style', 'commentary', 'description', 'labeltext', 'img_description', 'img_alttext', 'img_publiccaption']
for c in desc_list:
    harvard[c] = harvard[c].fillna('')

In [30]:
harvard['all_description'] = harvard[['style', 'commentary', 'description', 'labeltext', 
         'img_description', 'img_alttext', 'img_publiccaption']].agg(' '.join, axis=1)

In [32]:
harvard = harvard[['id', 'period', 'medium', 'title', 'culture', 'color', 'artist', 'all_description']]

In [33]:
harvard['source'] = 'harvard'

## MOMA
---
Now similar steps for MOMA data

### Missing image
drop missing images

In [34]:
moma = moma.dropna(subset = ['ThumbnailURL'])
moma = moma[moma.Classification == 'Painting']

In [35]:
moma['source'] = 'moma'

In [36]:
col_list = ['ObjectID', 'Title', 'Artist', 'Nationality', 'Date', 'Medium', 'source']
moma = moma[col_list]

## RISD
---

In [37]:
risd = pd.concat([risd1, risd2], axis = 0)

### Missing images
Remove if the image is missing.

In [38]:
risd = risd[[len(x) > 0 for x in risd['images']]]

### Location
Get the primary maker's location.

In [39]:
def get_nationality(x):
    '''
    given a dictionaries nested in a list, 
    return the first person's nationality
    '''
    if x:
        ind = list(x)[0]['nationality']
        if ind: 
            return ind[0]
        else: 
            return None


In [40]:
risd['nationality'] = risd.makers.apply(lambda x: get_nationality(x))

### Consolidating info
1. only paintings or other art-like work on paper 
2. culture: culture if empty, place, if empty nationality

In [41]:
# subsetting column
col_list = ['id', 'culture', 'dating', 'description', 'mediumTechnique', 'place', 'primaryMaker',  
            'title', 'type', 'nationality']
risd = risd[col_list]

In [42]:
# filter to only paintings
art_list = ['Paintings']
risd = risd[[any(item in x[0] for item in art_list) for x in risd['type']]]

#### Missing values
RISD datasets don't have missing values. They just have empty values. Let's change them.

In [43]:
risd = risd.replace('', np.nan)

#### Culture

In [44]:
# culture
risd['place'] = np.where(risd['place'].isnull(), risd['nationality'], risd['place'])
risd['culture'] = np.where(risd['culture'].isnull(), risd['place'], risd['culture'])
risd = risd.drop(['place', 'nationality'], axis = 1)

In [45]:
risd = risd.drop(['type'], axis = 1)

In [46]:
risd['source'] = 'risd'

# Purging
Now check if there is an actual matching image file, if not drop it.

In [47]:
import os

In [48]:
harvard_img_list = os.listdir('IMAGES/HARVARD')

In [49]:
def check_image(x, list_):
    '''
    return the filename if the file exists in list_
    otherwise np.nan
    '''
    fn = f'{x}.jpg' 
    if fn in list_:
        return fn
    else: return np.nan

In [50]:
harvard['image'] = harvard['id'].map(lambda x: check_image(x, harvard_img_list))

In [51]:
moma_img_list = os.listdir('IMAGES/MOMA')
moma['image'] = moma['ObjectID'].map(lambda x: check_image(x, moma_img_list))

In [52]:
def check_image_risd(x, list_):
    '''
    return the filename if the file exists in list_
    otherwise np.nan
    '''
    fn = f'risd_{x}.jpg' 
    if fn in list_:
        return fn
    else: return np.nan

In [53]:
risd_img_list = os.listdir('IMAGES/RISD')
risd['image'] = risd['id'].map(lambda x: check_image_risd(x, risd_img_list))

# Merging
now let's merge all dataframes

In [54]:
# first making consistent column names

In [55]:
harvard = harvard.rename(columns={'all_description': 'description'})

In [57]:
moma = moma.rename(columns = {'ObjectID': 'id', 'Title': 'title', 'Artist': 'artist', 'Nationality': 'culture', 
                             'Date': 'period', 'Medium': 'medium'})

In [58]:
risd = risd.rename(columns = {'dating': 'period', 'mediumTechnique': 'medium', 'primaryMaker': 'artist'})

In [406]:
full_df = pd.concat([harvard, risd, moma], ignore_index = True, sort = False)

In [407]:
full_df = full_df[~full_df.image.isnull()]

In [408]:
full_df.head(3)

,id,period,medium,title,culture,color,artist,description,source,image
0,47769,1644,Album of fourteen leaves; ink and color on silk,Page from an album of Rice and Silk Culture,Chinese,"Brown, Yellow",Traditionally attributed to Qiu Ying,,harvard,47769.jpg
1,47969,1644,Album of fourteen leaves; ink and color on silk,Page from an album of Rice and Silk Culture,Chinese,"Brown, Yellow, Green",Traditionally attributed to Qiu Ying,,harvard,47969.jpg
2,48085,"Qing dynasty, 1644-1911",Album leaf; ink and color on paper,Leaf from an Album: England and China,Chinese,"Green, Orange, Brown, Yellow",None,,harvard,48085.jpg


# Deep Cleaning
---
Now I'll clean the data collectively. Some of the things I need to do ...
1. Check for any abnormality in data
2. Consolidate inconsistent categorizations

In [409]:
# reset index
full_df = full_df.reset_index()

### Period
First into 00s. 

In [410]:
import re

In [411]:
# remove all non-digits
tmp = [re.sub('th', '00', str(x)) for x in full_df.period]

# for later, period names without numbers
# list of [x for x in tmp if re.match('^\D*$', x)]

# remove all non-digits
tmp = [re.sub('[^\d-]', '', str(x)) for x in tmp]
# remove all number after '-'
tmp = [re.sub('-.*', '', str(x)) for x in tmp]

# turn them into numbers
tmp = [int(x) if x else 0 for x in tmp]

In [412]:
tmp = [int(str(x)[0:4]) if x > 2030 else x for x in tmp ]

In [413]:
tmp = np.array(tmp)

In [414]:
# upto 1800, combine all century, after that, go by decades
cond = [(tmp < 1300) | (tmp > 2030), 
       tmp < 1900]
val = ['early', (tmp//100)*100]
full_df['period'] = np.select(cond, val, (tmp//10)*10)

## Medium
This would be multi-label case. 
I'll first consolidate all the unique mediums, categorize them and then turn them into a list of binary columns for each categories.

In [415]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# lemmatizer
wnl = WordNetLemmatizer()

# special chracters
special_chars = string.punctuation + string.digits

# stopwords
sw = stopwords.words('english')
sw += ['color', 'mounted', 'synthetic', 'hanging', 'painted', 'signature', 'reading', 'two', 'light', 'artist', 
      'one', 'opaque', 'colors', 'folding', 'three', 'one', 'frame', 'painting', 'parts', 'album', 'seal', 
       'nan', 'style', 'seals']

medium_text_clean = [x for x in medium_text.split() if x not in sw]

def text_preprocess(x):
    # remove punctuations and digits
    if isinstance(x, str):
        table_ = str.maketrans('', '', special_chars)
        text = str.translate(x, table_)
        text = text.split()
        clean_text = [wnl.lemmatize(w.lower()) for w in text]
        return [w for w in clean_text if w not in sw]
    else: 
        return np.nan

In [416]:
full_df['medium'] = full_df['medium'].apply(lambda x: text_preprocess(x))

In [417]:
# get frequencies
from collections import Counter
#top_30_medium = 
top_30_medium = list(dict(Counter(np.sum([x for x in full_df['medium'] if isinstance(x, list)])).most_common(30)).keys())

In [418]:
for med in top_30_medium: 
    full_df[med] = [str(med) in x if isinstance(x, list) else 'h' for x in full_df['medium']]

In [419]:
full_df['other_medium'] = np.where(np.sum(full_df.iloc[:, -30:], axis = 1) == 0, 1, 0)

## Artist
Now let's check artist info.

In [420]:
# lower case
full_df['artist'] = [str(x).lower() for x in full_df['artist']]

In [421]:
full_df['artist'] = full_df.artist.replace(['unknown artist', 'unidentified artist'], 'unknown')